In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 8]

# tk libs
from lib.objs.tkpaths import Site
from lib.data_fetcher.STORMS import Download_NOAA_WMO
from lib.tcyclone import Extract_Circle
from lib.plotting.storms import WorldMap_Storms
from lib.statistical import CopulaSimulation


# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_hist_tcs = DB.TCs.noaa

# output files
p_hist_r1 = ST.TCs.hist_r1
p_hist_r1_params = ST.TCs.hist_r1_params
p_hist_r2 = ST.TCs.hist_r2
p_hist_r2_params = ST.TCs.hist_r2_params

# wave point lon, lat and radius for TCs selection
pnt_lon = float(PR.WAVES.point_longitude)
pnt_lat = float(PR.WAVES.point_latitude)
r1 = float(PR.TCS.r1)   # bigger one
r2 = float(PR.TCS.r2)   # smaller one


In [2]:
# --------------------------------------
# Select Historical TCs inside circle

# Load historical TCs 
xds_wmo = xr.open_dataset(p_hist_tcs)

# dictionary with needed variable names 
d_vns = {
    'longitude': 'lon_wmo',
    'latitude': 'lat_wmo',
    'time': 'time_wmo',
    'pressure': 'pres_wmo',
}


# Select TCs that crosses a circular area R1
print(
'\nExtracting Historical TCs from WMO database...\n\
    Lon = {0:.2f}º , Lat = {1:.2f}º, R = {2:6.2f}º'.format(
    pnt_lon, pnt_lat, r1)
)

xds_TCs_r1, xds_TCs_r1_params = Extract_Circle(
    xds_wmo, pnt_lon, pnt_lat, r1, d_vns)

# store data
xds_TCs_r1.to_netcdf(p_hist_r1)
xds_TCs_r1_params.to_netcdf(p_hist_r1_params)

print('\nHistorical TCs selection and parameters stored at:\n{0}\n{1}'.format(
    p_hist_r1, p_hist_r1_params))


# Select TCs that crosses a circular area R2
print(
'\nExtracting Historical TCs from WMO database...\n\
    Lon = {0:.2f}º , Lat = {1:.2f}º, R = {2:6.2f}º'.format(
    pnt_lon, pnt_lat, r2)
)

xds_TCs_r2, xds_TCs_r2_params = Extract_Circle(
    xds_wmo, pnt_lon, pnt_lat, r2, d_vns)

# store data
xds_TCs_r2.to_netcdf(p_hist_r2)
xds_TCs_r2_params.to_netcdf(p_hist_r2_params)

print('\nHistorical TCs selection and parameters stored at:\n{0}\n{1}'.format(
    p_hist_r2, p_hist_r2_params))



Extracting Historical TCs from WMO database...
    Lon = 167.50º , Lat = 9.75º, R =  14.00º

Historical TCs selection and parameters stored at:
/Users/nico/Projects/TESLA-kit/source/teslakit/data/sites/KWAJALEIN/TCs/TCs_hist_r1.nc
/Users/nico/Projects/TESLA-kit/source/teslakit/data/sites/KWAJALEIN/TCs/TCs_hist_r1_params.nc

Extracting Historical TCs from WMO database...
    Lon = 167.50º , Lat = 9.75º, R =   4.00º

Historical TCs selection and parameters stored at:
/Users/nico/Projects/TESLA-kit/source/teslakit/data/sites/KWAJALEIN/TCs/TCs_hist_r2.nc
/Users/nico/Projects/TESLA-kit/source/teslakit/data/sites/KWAJALEIN/TCs/TCs_hist_r2_params.nc


In [2]:
# --------------------------------------
# Probabilistic simulation Historical TCs

# aux function
def FixPareto(var):
    'Fix data. It needs to start at 0 for Pareto adjustment '
    
    mx = np.amax(var)
    aux = mx + np.absolute(var - mx)
    var_pareto = aux - np.amin(aux) + 0.00001
    
    return var_pareto


# use small radius (4º)
xds_TCs_r2_params = xr.open_dataset(p_hist_r2_params)
pmean = xds_TCs_r2_params.pressure_mean.values[:]
pmin = xds_TCs_r2_params.pressure_min.values[:]
gamma = xds_TCs_r2_params.gamma.values[:]
delta = xds_TCs_r2_params.delta.values[:]
vmean = xds_TCs_r2_params.velocity_mean.values[:]

# fix pressure for pareto adjustment (start at 0)
pmean_p = FixPareto(pmean)
pmin_p = FixPareto(pmin)

# join storm parameters for copula simulation
storm_params = np.column_stack(
    (pmean_p, pmin_p, gamma, delta, vmean)
)
    
# statistical simulate PCs using copulas 
kernels = ['GPareto', 'GPareto', 'ECDF', 'ECDF', 'ECDF']
storm_params_sim = CopulaSimulation(storm_params, kernels, 10000) # TODO: NSIM PARAMETERS

print storm_params_sim


/Users/nico/Projects/TESLA-kit/source/venv/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:2159: RuntimeWarning: divide by zero encountered in true_divide
  val = val + cnk * (-1) ** ki / (1.0 - c * ki)
/Users/nico/Projects/TESLA-kit/source/venv/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1037: RuntimeWarning: invalid value encountered in subtract
  mu2 = mu2p - mu * mu
/Users/nico/Projects/TESLA-kit/source/venv/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:2305: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


[[ 26.94551151  30.49206301  95.51716826 152.57457351  23.05772123]
 [  8.06585503   7.73897667 135.89328252 165.66496292  14.47559437]
 [  9.09976916  11.83284152 104.94268743 152.39404609  17.65813967]
 ...
 [ 12.54555967  23.15313154  96.49952114  69.48878664  32.69116737]
 [  6.5740164    8.89239412 101.15601528  64.49261726  16.37870796]
 [ 24.42745739  35.7742277   96.96666716 112.64407696  22.9898277 ]]
